# LSTM STOCK 

In [ ]:
# 1. Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# 2. Generate synthetic stock data
np.random.seed(42)
time = np.arange(0, 500)
stock_prices = 50 + 0.1 * time + 5 * np.sin(0.1 * time) + np.random.normal(scale=2, size=len(time))

df = pd.DataFrame(stock_prices, columns=["Price"])

# 3. Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df)

# 4. Create sequences
def create_sequences(data, seq_length=50):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LEN = 50
X, y = create_sequences(scaled, SEQ_LEN)

# Reshape for LSTM [samples, time_steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# 5. Train/test split
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 6. Build LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(SEQ_LEN, 1)),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer="adam", loss="mse")
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# 7. Predictions
preds = model.predict(X_test)
preds = scaler.inverse_transform(preds)
y_test_rescaled = scaler.inverse_transform(y_test)

# 8. Plot results
plt.figure(figsize=(10, 6))
plt.plot(y_test_rescaled, label="True Prices")
plt.plot(preds, label="Predicted Prices")
plt.legend()
plt.title("Synthetic Stock Price Prediction with LSTM")
plt.show()
